In [1]:
from pyspark.sql import SparkSession

In [40]:
spark = SparkSession.builder.appName("Flipkar_Dataset").getOrCreate()

In [55]:
fp_df = spark.read.csv('Flipkart.csv', header=True, inferSchema=True)
fp_df.show(5)

+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| 2242|Casuals For Men  ...|   3.8|      Men|Flipkart|      999|    27928|      3543|  14238|   4295|   3457|   1962|   3976|         1|
|20532|Women Black Flats...|   3.9|    Women|Flipkart|      499|     3015|       404|   1458|    657|    397|    182|    321|         1|
|10648|Women Gold Wedges...|   3.9|    Women|Flipkart|      999|      449|        52|    229|     70|     71|     33|     46|         1|
|20677|Men's Height Incr...|   3.9|      Men|Flipkart|     2999|      290|        40|    141|     51|     49|     17|     32|         1|
|12593|Loafers For Men  ...|   3.9|      

In [47]:
fp_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- maincateg: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- actprice1: integer (nullable = true)
 |-- norating1: integer (nullable = true)
 |-- noreviews1: integer (nullable = true)
 |-- star_5f: integer (nullable = true)
 |-- star_4f: integer (nullable = true)
 |-- star_3f: integer (nullable = true)
 |-- star_2f: integer (nullable = true)
 |-- star_1f: integer (nullable = true)
 |-- fulfilled1: integer (nullable = true)



In [62]:
from pyspark.sql.functions import col, isnan, isnull, when, count, expr

In [71]:
from pyspark.sql.functions import *

In [57]:
## handling the missing data

fp_df_null = fp_df.select([count(when(col(c).isNull(), c)).alias(c) for c in fp_df.columns])
fp_df_null.show()

+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| id|title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|  0|    0|   203|       67|       0|        0|        0|         0|     68|      0|      0|      0|    186|         0|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+



In [50]:
fp_df_null = fp_df.select([count(when(col(c).isNull(), c)).alias(c) for c in fp_df.columns])
fp_df_null.show()

+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| id|title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|  0|    0|   203|       67|       0|        0|        0|         0|     68|      0|      0|      0|    186|         0|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+



In [60]:
##Drop null value
fp_df_drop = fp_df.na.drop()

##fill missing value
fp_df_fill = fp_df.fillna({
                    'Rating' : 0
})
fp_df_fill.show()

+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| 2242|Casuals For Men  ...|   3.8|      Men|Flipkart|      999|    27928|      3543|  14238|   4295|   3457|   1962|   3976|         1|
|20532|Women Black Flats...|   3.9|    Women|Flipkart|      499|     3015|       404|   1458|    657|    397|    182|    321|         1|
|10648|Women Gold Wedges...|   3.9|    Women|Flipkart|      999|      449|        52|    229|     70|     71|     33|     46|         1|
|20677|Men's Height Incr...|   3.9|      Men|Flipkart|     2999|      290|        40|    141|     51|     49|     17|     32|         1|
|12593|Loafers For Men  ...|   3.9|      

In [68]:
fp_df_fill.filter(fp_df_fill.Rating == 0).show()

+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|17580|Women Beige Heels...|   0.0|    Women|Flipkart|      999|      473|        68|   NULL|    119|     45|     16|   NULL|         1|
|11588|Royal Jutis For m...|   0.0|     NULL|Flipkart|      899|       41|         7|     22|      9|      3|      1|      6|         0|
|15449|          Flip Flops|   0.0|     NULL|Flipkart|      499|     3073|       401|   1248|    517|    510|    274|   NULL|         1|
|12342|Men Black, Blue S...|   0.0|      Men|Flipkart|     2499|       52|         5|   NULL|     13|      6|      4|   NULL|         1|
| 9065|Sneakers For Men ...|   0.0|      

In [69]:
high_rated_product = fp_df_fill.filter('Rating > 4')
high_rated_product.show()

+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| 6433|ARYA - DIFFERENT ...|   4.2|    Women|Flipkart|     4299|      166|        24|     94|     39|     12|      6|     15|         1|
|13859|Women Black Wedge...|   4.2|    Women|Flipkart|      279|     3048|       487|   1746|    679|    348|    114|    161|         1|
|19453|Denill Ankle Leng...|   4.1|    Women|Flipkart|      999|     6806|       961|   3646|   1508|    810|    335|    510|         1|
| 8121|Women Grey Heels ...|   4.2|    Women|Flipkart|     1990|       16|         1|     10|      2|      1|      3|      0|         1|
| 9791|Pink Perfect Styl...|   4.2|    Wo

In [70]:
## group by and categeory wise avg rating

avg_rating = fp_df_drop.groupBy('maincateg').avg('Rating')
avg_rating.show()

+---------+-----------------+
|maincateg|      avg(Rating)|
+---------+-----------------+
|      Men|3.967410281280314|
|    Women|4.041507503410653|
+---------+-----------------+



In [72]:
## total rating by categeory wise

avg_rating_total = fp_df_drop.groupBy('maincateg').agg(sum('Rating'))
avg_rating_total.show()

+---------+------------------+
|maincateg|       sum(Rating)|
+---------+------------------+
|      Men|8180.8000000000075|
|    Women|11849.700000000035|
+---------+------------------+



In [75]:
output_table = 'Flipkart_data_analysis'
fp_df_drop.write.mode('overwrite').saveAsTable(output_table)

In [77]:
%sqrt
select * from Flipkart_data_analysis limit 20;

SyntaxError: invalid syntax (2898414346.py, line 2)